In [1]:
!pip install neo4j


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import glob

with open("elt_cypher_queries/cleanup_nodes.cypher") as f:
    cleanup_queries = [line for line in f]

with open("elt_cypher_queries/index_nodes.cypher") as f:
    index_queries = [line for line in f]

node_cypher_files = glob.glob("elt_cypher_queries/nodes/*.cypher")
creation_queries = []
for query_file in node_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        creation_queries.append(creation_query)

In [12]:
from neo4j import GraphDatabase

class IMDBELTLoader:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def execute_query(self, query, params_dict={}):
        with self.driver.session(database="neo4j") as session:
            result = session.run(query, **params_dict)
            print(result.consume().counters)

for query in cleanup_queries+creation_queries+index_queries:
    print(f"Processing {query}")
    loader = IMDBELTLoader("bolt://localhost:7687", "neo4j", "pass1234")
    loader.execute_query(query)
    loader.close()


Processing DROP INDEX imdb_attribute_attribute IF EXISTS;

{}
Processing DROP INDEX imdb_characters_somecharacter IF EXISTS;

{}
Processing DROP INDEX imdb_distribution_type_distribution_type IF EXISTS;

{}
Processing DROP INDEX imdb_genre_genre IF EXISTS;

{}
Processing DROP INDEX imdb_job_category_category IF EXISTS;

{}
Processing DROP INDEX imdb_language_code_alpha2 IF EXISTS;

{}
Processing DROP INDEX imdb_name_basics_nconst IF EXISTS;

{}
Processing DROP INDEX imdb_primaryprofession_profession IF EXISTS;

{}
Processing DROP INDEX imdb_region_code_alpha2 IF EXISTS;

{}
Processing DROP INDEX imdb_title_akas_id IF EXISTS;

{}
Processing DROP INDEX imdb_title_basics_tconst IF EXISTS;

{}
Processing DROP INDEX imdb_title_episode_epitconst IF EXISTS;

{}
Processing DROP INDEX imdb_title_principals_id IF EXISTS;

{}
Processing DROP INDEX imdb_title_ratings_tconst IF EXISTS;

{}
Processing DROP INDEX imdb_title_type_title_type IF EXISTS;

{}
Processing MATCH (n) CALL { WITH n DETACH DELE